# Applied ML CSC 4505 Final Project 

### Yasmin Lorin Kaygalak & Natalie Bohmbach



In [ ]:
!pip install seaborn
!python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.9 MB/s eta 0:00:00 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Importing necessary libraries
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

# Downloading necessary data
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lorin.kaygalak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lorin.kaygalak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lorin.kaygalak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# CONFIG
# Change this to False if you want to keep stopwords
REMOVE_STOPWORDS = True

In [35]:
# STEP 1
# Reading fake news dataset into a pandas dataframe
fake_df = pd.read_csv('datasets/Fake.csv')
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [36]:
# STEP 1
# Reading true news dataset into a pandas dataframe
true_df = pd.read_csv('datasets/True.csv')
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [37]:
# STEP 2
# Adding labels 
fake_df['label'] = 0  # Fake news
true_df['label'] = 1  # Real news

In [38]:
# STEP 3
# Checking the info for null values - neither dataset has any
fake_df.info()
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
 4   label    23481 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 917.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
 4   label    21417 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 836.7+ KB


In [39]:
# STEP 4
# Combining datasets for unified training
df = pd.concat([fake_df, true_df], ignore_index=True)

In [43]:
# STEP 5
# Combine title and text
df['combined'] = df['title'].astype(str) + " " + df['text'].astype(str)

# Cleaning and tokenizing text (and optionally remove stopwords) 
def spacy_tokenizer_pipe(docs, remove_stopwords=True):
    results = []
    for doc in nlp.pipe(docs, batch_size=100):
        tokens = [token.text.lower() for token in doc if token.is_alpha]
        if remove_stopwords:
            tokens = [word for word in tokens if word not in stop_words]
        results.append(tokens)
    return results

# Apply spaCy tokenizer in batch mode
df['text_tokens'] = spacy_tokenizer_pipe(df['combined'], remove_stopwords=REMOVE_STOPWORDS)
df['text_clean'] = df['text_tokens'].apply(lambda tokens: ' '.join(tokens))


KeyboardInterrupt: 

In [ ]:
# STEP 6
# Joining tokens back into string for TF-IDF
df['text_clean'] = df['text_tokens'].apply(lambda tokens: ' '.join(tokens))

In [ ]:
# STEP 7
# Train and Test data split 
X = df['text_clean']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# STEP 8
# TF-IDF Vectorization 
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# STEP 9 Defining Models 
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "SVM (Linear Kernel)": LinearSVC(),
    "MLP Classifier": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
}

In [41]:
# STEP 10
# Train, Predict, and Evaluate 
for name, model in models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=["Fake", "Real"]))

    # Confusion Matrix 
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Fake", "Real"], yticklabels=["Fake", "Real"])
    plt.title(f"{name} - Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

NameError: name 'models' is not defined